In [106]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt

In [26]:
data = pd.read_csv('https://raw.githubusercontent.com/akash9579/work/master/covid/datasets_549966_1296705_complete.csv')

In [27]:
data.head(1)

,Date,Name of State / UT,Latitude,Longitude,Total Confirmed cases,Death,Cured/Discharged/Migrated,New cases,New deaths,New recovered
0,2020-01-30,Kerala,10.8505,76.2711,1,0,0,0,0,0


In [28]:
datewise=data.groupby("Date")["Total Confirmed cases", "Cured/Discharged/Migrated", "Death"].sum()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [29]:
datewise.head(5)

,Total Confirmed cases,Cured/Discharged/Migrated,Death
Date,,,
2020-01-30,1,0,0
2020-01-31,1,0,0
2020-02-01,2,0,0
2020-02-02,3,0,0
2020-02-03,3,0,0


In [30]:
date=datewise.index.tolist()

In [31]:
datewise = datewise.assign(date = date)

In [50]:
datewise['date']= pd.to_datetime(datewise['date'])

In [84]:
datewise.info()

<class 'pandas.core.frame.DataFrame'>
Index: 152 entries, 2020-01-30 to 2020-06-30
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Total Confirmed cases      152 non-null    int64         
 1   Cured/Discharged/Migrated  152 non-null    int64         
 2   Death                      152 non-null    int64         
 3   date                       152 non-null    datetime64[ns]
 4   day                        152 non-null    int64         
 5   month                      152 non-null    int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 8.3+ KB


In [69]:
sr=datewise['date']

In [70]:
srday = sr.dt.day 

In [71]:
srmonth = sr.dt.month

In [72]:
type(srmonth)

pandas.core.series.Series

In [73]:
datewise = datewise.assign(day = srday)
datewise = datewise.assign(month = srmonth)

In [74]:
datewise.head(5)

,Total Confirmed cases,Cured/Discharged/Migrated,Death,date,day,month
Date,,,,,,
2020-01-30,1,0,0,1970-01-01 00:00:00.020200130,1,1
2020-01-31,1,0,0,2020-01-31 00:00:00.000000000,31,1
2020-02-01,2,0,0,2020-02-01 00:00:00.000000000,1,2
2020-02-02,3,0,0,2020-02-02 00:00:00.000000000,2,2
2020-02-03,3,0,0,2020-02-03 00:00:00.000000000,3,2


In [75]:
fig=px.bar(x=datewise.index,y=datewise["Total Confirmed cases"]-datewise["Cured/Discharged/Migrated"]-datewise["Death"])
fig.update_layout(title="Distribution of Number of Active Cases",
                  xaxis_title="Date",yaxis_title="Number of Cases",)
fig.show()

In [76]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=datewise.index, y=datewise["Total Confirmed cases"],
                    mode='lines+markers',
                    name='Confirmed Cases'))
fig.add_trace(go.Scatter(x=datewise.index, y=datewise["Cured/Discharged/Migrated"],
                    mode='lines+markers',
                    name='Recovered Cases'))
fig.add_trace(go.Scatter(x=datewise.index, y=datewise["Death"],
                    mode='lines+markers',
                    name='Death Cases'))
fig.update_layout(title="Growth of different types of cases",
                 xaxis_title="Date",yaxis_title="Number of Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()



In [77]:
train_ml=datewise.iloc[:int(datewise.shape[0]*0.95)]
valid_ml=datewise.iloc[int(datewise.shape[0]*0.95):]
model_scores=[]

In [104]:
x2 = datewise.loc[:,['day','month']]
y2 = datewise['Total Confirmed cases']


In [78]:
train_ml.head()
# we can reset index aswell if neccessary

,Total Confirmed cases,Cured/Discharged/Migrated,Death,date,day,month
Date,,,,,,
2020-01-30,1,0,0,1970-01-01 00:00:00.020200130,1,1
2020-01-31,1,0,0,2020-01-31 00:00:00.000000000,31,1
2020-02-01,2,0,0,2020-02-01 00:00:00.000000000,1,2
2020-02-02,3,0,0,2020-02-02 00:00:00.000000000,2,2
2020-02-03,3,0,0,2020-02-03 00:00:00.000000000,3,2


In [101]:
y = train_ml['Total Confirmed cases']

In [91]:
x.shape

(144, 2)

In [87]:
x = train_ml.loc[:,['day','month']]

In [93]:
x1 = valid_ml.loc[:,['day','month']]

In [102]:
y1 = valid_ml['Total Confirmed cases']

In [126]:
lin_reg=LinearRegression()

In [127]:
lin_reg.fit(x, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [128]:
prediction_valid_linreg=lin_reg.predict(x1)

In [129]:
prediction_valid_linreg

array([200068.68361563, 201201.14863636, 202333.61365709, 203466.07867782,
       205731.00871928, 206863.47374001, 207995.93876075, 209128.40378148])

In [130]:
model_scores.append(np.sqrt(mean_squared_error(y1,prediction_valid_linreg)))
print("Root Mean Square Error for Linear Regression: ",np.sqrt(mean_squared_error(y1,prediction_valid_linreg)))

Root Mean Square Error for Linear Regression:  284809.40535643935


In [131]:
plt.figure(figsize=(11,6))
prediction_linreg=lin_reg.predict(x2)
linreg_output=[]
for i in range(prediction_linreg.shape[0]):
    linreg_output.append(prediction_linreg[i])

fig=go.Figure()
fig.add_trace(go.Scatter(x=datewise.index, y=datewise["Total Confirmed cases"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=datewise.index, y=prediction_linreg,
                    mode='lines',name="Linear Regression Best Fit Line",
                    line=dict(color='black', dash='dot')))
fig.update_layout(title="Confirmed Cases Linear Regression Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

<Figure size 792x432 with 0 Axes>

In [115]:
from sklearn.preprocessing import PolynomialFeatures
polynomial_features= PolynomialFeatures(degree=2)
x_poly = polynomial_features.fit_transform(x)

In [116]:
x_poly1 = polynomial_features.fit_transform(x1)

In [121]:
x_poly2 = polynomial_features.fit_transform(x2)

In [117]:
model = LinearRegression()
model.fit(x_poly, y)
y_poly_pred = model.predict(x_poly1)

In [118]:
y_poly_pred

array([324460.40527182, 329072.07728684, 333536.41437874, 337853.41654752,
       346045.41611572, 349920.41351515, 353648.07599145, 357228.40354464])

In [119]:
model_scores.append(np.sqrt(mean_squared_error(y1,y_poly_pred)))
print("Root Mean Square Error for Linear Regression: ",np.sqrt(mean_squared_error(y1,y_poly_pred)))

Root Mean Square Error for Linear Regression:  149117.8022133794


In [122]:
final = model.predict(x_poly2)

In [ ]:
plt.figure(figsize=(11,6))
prediction_linreg=lin_reg.predict(x2)
linreg_output=[]
for i in range(prediction_linreg.shape[0]):
    linreg_output.append(prediction_linreg[i])

fig=go.Figure()
fig.add_trace(go.Scatter(x=datewise.index, y=datewise["Total Confirmed cases"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=datewise.index, y=prediction_linreg,
                    mode='lines',name="Linear Regression Best Fit Line",
                    line=dict(color='black', dash='dot')))
fig.update_layout(title="Confirmed Cases Linear Regression Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [125]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=datewise.index, y=datewise["Total Confirmed cases"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=datewise.index, y=final,
                    mode='lines',name="Linear Regression Best Fit Line",
                    line=dict(color='black', dash='dot')))
fig.update_layout(title="Confirmed Cases Linear Regression Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()